<a href="https://colab.research.google.com/github/GianFadiga/MCDA_FS/blob/main/mcda_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/GianFadiga/MCDA_FS/blob/main/MCDA_UT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# THE MCDA PROJECT: Criação da Ferramenta

## Início

#### Vamos ao código!

In [80]:
'''
  Aqui toda a execução do código se inicia, não se esqueça de executar
  esta célula que é fundamental para o funcionamento do código
'''

# Importar as bibliotecas a serem utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [81]:
'''
  DICA: Divida os dados qualitativos em categorias (exemplo na CÉLULA 2),
  aqui está uma tabela para auxiliar no processo.
'''

# - Dividir em 5 categorias
# (0 - 20% (Nada desejável),
# 20 - 40% (Pouco desejável),
# 40 - 60% (Médio desejável),
# 60 - 80% (Bem desejável),
# 80 - 100% (Muito desejável))

'\n  DICA: Divida os dados qualitativos em categorias (exemplo na CÉLULA 2),\n  aqui está uma tabela para auxiliar no processo.\n'

# Realizando a montagem dos gráficos

## IDEIA ORIGINAL

In [ ]:
df = pd.DataFrame()

# Supondo a compra de um notebook (altere como desejar)

'''
  Instruções de uso:
    - Cada linha (ex: df['Modelo']) representa uma coluna, portanto, preencha
    cada item referente a cada uma das colunas de forma sequencial conforme mostrado.

    - As tabelas geradas se referem unica e somente a dados numéricos, logo,
    para geração dos dados qualitativos será necessário executar a célula identificada
    como (CÉLULA 2) uma a uma conforme a quantidade de dados qualitativos inseridos no
    dataframe. O programa assume como 'object' dados qualitativos, sendo assim, é
    fácil saber diferenciar e preencher os dados para a CÉLULA 2.
'''

# Tabela principal de dados
# Altere aqui! - Início
df['Modelo'] = ['Acer', 'Lenovo', 'DELL', 'Samsung', 'Positivo']
df['Memória RAM'] = [2, 4, 6, 8, 16]
df['Armazenamento'] = [960, 480, 480, 240, 120]
df['Tamanho de tela'] = [14, 15, 15, 14, 16]
df['Bateria mAh'] = [4000, 3000, 3000, 4000, 5000]
df['Cor'] = ['Preto', 'Prata', 'Vermelho', 'Azul', 'Verde']

# Tabela dos classificadores

classificadores = pd.DataFrame()

classificadores['Modelo'] = ['BOM', 'NEUTRO']
classificadores['Memória RAM'] = [8, 4]
classificadores['Armazenamento'] = [960, 120]
classificadores['Tamanho de tela'] = [14, 14]
classificadores['Bateria mAh'] = [5000, 3000]
classificadores['Cor'] = ['Prata', 'Preto']

# Altere aqui! - Final

# A linha abaixo faz a concatenação dos valores novos referentes a BOM e NEUTRO
df = pd.concat([df, classificadores], ignore_index=True)

# Definindo a tabela auxiliar que fará os gráficos

df_analise = pd.DataFrame()

df_analise['Montante Relacional'] = pd.Series(dtype='int')

for i in range(len(df)-1):
    aux_montanteRelacional = df.columns.values[i]
    val_neutro = df[aux_montanteRelacional][df.index[df['Modelo'] == 'NEUTRO'][0]]
    if df.dtypes[i] == 'int64':

        for i in range (len(df)):
            df.loc[i, 'Montante Relacional'] = df[aux_montanteRelacional][i]/val_neutro
            if (df['Montante Relacional'][i] == val_neutro).all() & (df['Montante Relacional'] <= val_neutro).all():
                df.loc[i, 'Montante Relacional'] = 0
            elif df['Montante Relacional'][i] < 1:
                df.loc[i, 'Montante Relacional'] = df['Montante Relacional'][i] * (-1)
            else:
                df.loc[i, 'Montante Relacional'] = (df['Montante Relacional'][i])-1

        df_analise['Montante Relacional'] = df['Montante Relacional'].astype('float')
        df_analise['Modelo'] = df['Modelo'].astype('object')
        df.drop(['Montante Relacional'], axis=1, inplace=True)

        for i in range(len(df)):
            if df['Modelo'][i] == 'NEUTRO' or df['Modelo'][i] == 'BOM':
                df_analise.loc[i, 'Color'] = 'Classificador'
            else:
                df_analise['Color'] = 'Desempenho'


        cores1 = ['red', 'blue']
        cores2 = ['pink', 'green']
        cores3 = ['orange', 'purple']
        cores4 = ['orange', 'gray']

        df_analise['Porcentagem desempenho'] = pd.Series(dtype='str')
        for i in range (len(df)):
            df_analise.loc[i, 'Porcentagem desempenho'] = ((df_analise['Montante Relacional'][i]) * 100).astype(int).astype(str) + '%'

        graph = px.bar(df_analise, x='Montante Relacional', y='Modelo', orientation='h',
                    color='Color', color_discrete_sequence=cores1, text_auto=False, hover_name='Color',
                    hover_data={'Color':False, 'Modelo':False, 'Montante Relacional':False,
                                'Porcentagem desempenho': True, 'Cor indicativa': (df_analise['Color'])})

        graph.update_layout(yaxis={'categoryorder':'total ascending'})
        graph.update_xaxes(showticklabels=False)
        graph.update_yaxes(showticklabels=True)
        graph.update_coloraxes(showscale=False)
        graph.add_vline(x=0)
        graph.add_annotation(
                x=0,
                # y é definido como o tamanho do array, para que a linha fique no meio e em altura suficiente
                y=len(df_analise),
                xref="x",
                yref="y",
                text=f"Linha Neutra<br>(Mínimo Aceitável): {val_neutro}",
                showarrow=True,
                font=dict(
                    family="Courier New, monospace",
                    size=16,
                    color="#ffffff"
                    ),
                align="center",
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor="#636363",
                ax=0,
                ay=-45,
                bordercolor="#c7c7c7",
                borderwidth=2,
                borderpad=4,
                bgcolor=cores1[1],
                opacity=1
                )

        graph.update_layout(title='Comparação com Indicadores de Desempenho e Classificadores: {}'.format(aux_montanteRelacional),
                            legend_title_text='Legenda da relação de cores')
        graph.update_layout(hovermode='y unified')

        graph.show();

    else:
        print('Object')

In [4]:
import pandas as pd
import plotly.express as px

# Leitura do arquivo CSV com a codificação ISO-8859-1
teste_df = pd.read_csv('base_teste.csv', encoding='ISO-8859-1')

# Extraindo o montante relacional
montante_relacional = teste_df.iloc[0].drop('Modelo')
teste_df = teste_df.drop(0).reset_index(drop=True)

# Convertendo colunas numéricas para float
numeric_cols = teste_df.select_dtypes(include=['number']).columns
teste_df[numeric_cols] = teste_df[numeric_cols].astype(float)

# Definindo a tabela auxiliar que fará os gráficos
df_analise = pd.DataFrame()

# Adicionando a coluna 'Montante Relacional'
df_analise['Montante Relacional'] = pd.Series(dtype='float')

# Loop através das colunas de interesse (excluindo 'Modelo' e 'Cor')
for col in numeric_cols:
    val_neutro = teste_df[col][teste_df['Modelo'] == 'NEUTRO'].values[0]

    if teste_df[col].dtype == 'float64':
        for i in range(len(teste_df)):
            df_analise.loc[i, 'Montante Relacional'] = teste_df[col][i] / val_neutro

            if df_analise.loc[i, 'Montante Relacional'] == val_neutro and df_analise.loc[i, 'Montante Relacional'] <= val_neutro:
                df_analise.loc[i, 'Montante Relacional'] = 0
            elif df_analise.loc[i, 'Montante Relacional'] < 1:
                df_analise.loc[i, 'Montante Relacional'] *= -1
            else:
                df_analise.loc[i, 'Montante Relacional'] = df_analise.loc[i, 'Montante Relacional'] - 1

        df_analise[col] = df_analise['Montante Relacional'] * float(montante_relacional[col])

df_analise['Modelo'] = teste_df['Modelo']

# Identificando classificadores e desempenho
df_analise['Color'] = df_analise['Modelo'].apply(lambda x: 'Classificador' if x in ['BOM', 'NEUTRO'] else 'Desempenho')

# Somando os montantes relacionais ponderados
df_analise['Total Relacional'] = df_analise[numeric_cols].sum(axis=1)

# Gerando a coluna de porcentagem de desempenho
df_analise['Porcentagem desempenho'] = (df_analise['Total Relacional'] * 100).astype(int).astype(str) + '%'

# Encontrando o notebook com maior pontuação
recommended_notebook = df_analise[df_analise['Color'] == 'Desempenho'].sort_values(by='Total Relacional', ascending=False).iloc[0]

# Exibindo a recomendação
print(f"Notebook recomendado: {recommended_notebook['Modelo']} com pontuação total de {recommended_notebook['Total Relacional']}")

# Plotando o gráfico
graph = px.bar(df_analise, x='Total Relacional', y='Modelo', orientation='h',
               color='Color', color_discrete_sequence=['red', 'blue'], text_auto=False, hover_name='Color',
               hover_data={'Color': False, 'Modelo': False, 'Total Relacional': False, 'Porcentagem desempenho': True})

graph.update_layout(yaxis={'categoryorder': 'total ascending'})
graph.update_xaxes(showticklabels=False)
graph.update_yaxes(showticklabels=True)
graph.update_coloraxes(showscale=False)
graph.add_vline(x=0)
graph.add_annotation(
    x=0,
    y=len(df_analise),
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#ffffff"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='blue',
    opacity=1
)

graph.update_layout(title='Comparação com Indicadores de Desempenho e Classificadores',
                    legend_title_text='Legenda da relação de cores')
graph.update_layout(hovermode='y unified')

graph.show()

Notebook recomendado: nan com pontuação total de 0.0


## **CÉLULA 1 - DADOS NUMÉRICOS**

In [116]:
import pandas as pd
import plotly.express as px

# Leitura do arquivo CSV com a codificação ISO-8859-1
teste_df = pd.read_csv('base_expandida.csv', encoding='ISO-8859-1')

# Extraindo o montante relacional e os tipos de dados
montante_relacional = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
teste_df = teste_df.drop([0, 1]).reset_index(drop=True)

# Separando os valores 'BOM' e 'NEUTRO'
valores_bom = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')

# Convertendo apenas as colunas numéricas para float
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom[col] = float(valores_bom[col])
        valores_neutro[col] = float(valores_neutro[col])
        teste_df[col] = teste_df[col].astype(float)

# Função para calcular a pontuação baseada na proximidade ao neutro e ao bom
def calcular_pontuacao(valor, bom, neutro, peso):
    if bom - neutro == 0:
        return 0
    elif valor >= neutro:
        return ((valor - neutro) / (bom - neutro)) * peso
    else:
        return -((neutro - valor) / neutro) * peso

# Aplicando a função de pontuação para cada atributo numérico
for col in montante_relacional.index:
    if tipos_dados[col] == 'number':
        teste_df[col + '_pontuacao'] = teste_df.apply(lambda x: calcular_pontuacao(x[col], valores_bom[col], valores_neutro[col], montante_relacional[col]), axis=1)

# Somando as pontuações para obter a pontuação total
teste_df['Pontuacao_Total'] = teste_df[[col + '_pontuacao' for col in montante_relacional.index if tipos_dados[col] == 'number']].sum(axis=1)

# Adicionando classificadores 'BOM' e 'NEUTRO' ao DataFrame
teste_df.loc[teste_df['Modelo'] == 'BOM', 'Color'] = 'Classificadores'
teste_df.loc[teste_df['Modelo'] == 'NEUTRO', 'Color'] = 'Classificadores'
teste_df.loc[~teste_df['Modelo'].isin(['BOM', 'NEUTRO']), 'Color'] = 'Desempenho'

# Ordenando os produtos pela pontuação total
teste_df = teste_df.sort_values(by='Pontuacao_Total', ascending=False).reset_index(drop=True)

# Identificando a pontuação máxima entre os produtos recomendados
pontuacao_maxima = teste_df[teste_df['Color'] == 'Desempenho']['Pontuacao_Total'].max()
produtos_recomendados = teste_df[(teste_df['Color'] == 'Desempenho') & (teste_df['Pontuacao_Total'] == pontuacao_maxima)]

# Exibindo todos os produtos recomendados com detalhes para leigos
for _, produto in produtos_recomendados.iterrows():
    print(f"Produto recomendado: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
    print("Pontos de vantagem e justificativas:")

    pontuacoes_detalhadas = []
    for col in montante_relacional.index:
        if tipos_dados[col] == 'number':
            pontuacao_atributo = produto[col + '_pontuacao']
            if pontuacao_atributo > 0:
                # Contexto adicional para leigos
                justificativa = (
                    f"O produto teve vantagem no atributo '{col}' onde o desempenho neste atributo está {pontuacao_atributo * 100:.0f}% mais próximo do ideal ('BOM') "
                    f"em comparação com o mínimo aceitável ('NEUTRO')."
                )
                pontuacoes_detalhadas.append(justificativa)

    # Exibindo as justificativas detalhadas para o produto
    for detalhe in pontuacoes_detalhadas:
        print(detalhe)
    print("\n")

# Preparando os dados para o gráfico
df_analise = teste_df.copy()

# Definindo cores baseadas nas pontuações
# df_analise['Color'] = df_analise['Pontuacao_Total'].apply(
#     lambda x: 'Classificadores' if x >= 0 else 'Desempenho'
# )

# Definindo as cores com base na pontuação total usando lambda
df_analise['Color'] = df_analise.apply(
    lambda row: 'Classificadores' if row['Modelo'] in ['BOM', 'NEUTRO'] else ('Pontuação Positiva' if row['Pontuacao_Total'] >= 0 else 'Pontuação Negativa'),
    axis=1
)

# Definindo o mapeamento de cores
color_mapping = {
    'Classificadores': 'green',       # Para BOM e NEUTRO
    'Pontuação Positiva': 'blue',  # Para Desempenho
    'Pontuação Negativa': 'red'   # Para pontuação negativa
}

# Mapeando as cores corretamente
df_analise['Color'] = df_analise['Color'].map(color_mapping)

# Plotando o gráfico
graph = px.bar(df_analise, x='Pontuacao_Total', y='Modelo', orientation='h',
               color='Color', text_auto=False, hover_name='Color',
               hover_data={'Color': False, 'Modelo': False, 'Pontuacao_Total': True},
               color_discrete_sequence=["cornflowerblue", "lightgreen", "palevioletred"])

graph.update_layout(yaxis={'categoryorder': 'total ascending'})
graph.update_xaxes(showticklabels=True)
graph.update_yaxes(showticklabels=True)
graph.update_coloraxes(showscale=False)
graph.update_layout(xaxis_title='')

# Adicionando a linha neutra
graph.add_vline(x=0, line_dash='dash', line_color='black')
graph.add_annotation(
    x=0,
    y=len(df_analise),
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#000000"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='white',
    opacity=1
)

graph.update_layout(title='Comparação com pontuação final',
                    legend_title_text='Legenda da relação de cores')
graph.update_layout(hovermode='y unified')

graph.show()


Produto recomendado: Clevo com pontuação total de 5.45
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' onde o desempenho neste atributo está 450% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' onde o desempenho neste atributo está 95% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').


Produto recomendado: Eurocom com pontuação total de 5.45
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' onde o desempenho neste atributo está 450% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' onde o desempenho neste atributo está 95% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').




In [90]:
print(df_analise[['Modelo', 'Color']])

           Modelo Color
0           Clevo  blue
1         Eurocom  blue
2        Asus ROG  blue
3   Alienware M15  blue
4           Aorus  blue
..            ...   ...
63      Panasonic  blue
64  Gateway Ultra   red
65         Philco   red
66            TCL   red
67        Quantum   red

[68 rows x 2 columns]


In [89]:
# Filtrando as linhas onde a coluna 'Modelo' é 'BOM'
df_bom = df_analise[df_analise['Modelo'] == 'BOM']

# Imprimindo o DataFrame filtrado
df_bom.head()

,Modelo,Memória RAM,Armazenamento,Tamanho de tela,Bateria mAh,Cor,Memória RAM_pontuacao,Armazenamento_pontuacao,Tamanho de tela_pontuacao,Pontuacao_Total,Color
22,BOM,8.0,960.0,14.0,5000,Prata,0.3,0.2,0,0.5,green


In [39]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Leitura do arquivo CSV com a codificação ISO-8859-1
teste_df = pd.read_csv('base_expandida.csv', encoding='ISO-8859-1')

# Extraindo o montante relacional e os tipos de dados
montante_relacional = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
teste_df = teste_df.drop([0, 1]).reset_index(drop=True)

# Separando os valores 'BOM' e 'NEUTRO'
valores_bom = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')

# Convertendo apenas as colunas numéricas para float
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom[col] = float(valores_bom[col])
        valores_neutro[col] = float(valores_neutro[col])
        teste_df[col] = teste_df[col].astype(float)

# Função para calcular a pontuação com penalização negativa
def calcular_pontuacao(valor, bom, neutro, peso):
    if bom - neutro == 0:
        return 0
    elif valor >= neutro:
        return ((valor - neutro) / (bom - neutro)) * peso
    else:
        return -((neutro - valor) / neutro) * peso

# Aplicando a função de pontuação para cada atributo numérico
for col in montante_relacional.index:
    if tipos_dados[col] == 'number':
        teste_df[col + '_pontuacao'] = teste_df.apply(lambda x: calcular_pontuacao(x[col], valores_bom[col], valores_neutro[col], montante_relacional[col]), axis=1)

# Somando as pontuações para obter a pontuação total
teste_df['Pontuacao_Total'] = teste_df[[col + '_pontuacao' for col in montante_relacional.index if tipos_dados[col] == 'number']].sum(axis=1)

# Adicionando classificadores 'BOM' e 'NEUTRO' ao DataFrame
teste_df.loc[teste_df['Modelo'] == 'BOM', 'Color'] = 'BOM'
teste_df.loc[teste_df['Modelo'] == 'NEUTRO', 'Color'] = 'NEUTRO'
teste_df.loc[~teste_df['Modelo'].isin(['BOM', 'NEUTRO']), 'Color'] = 'Desempenho'

# Ordenando os produtos pela pontuação total
teste_df = teste_df.sort_values(by='Pontuacao_Total', ascending=False).reset_index(drop=True)

# Identificando a pontuação máxima entre os produtos recomendados
pontuacao_maxima = teste_df[teste_df['Color'] == 'Desempenho']['Pontuacao_Total'].max()
produtos_recomendados = teste_df[(teste_df['Color'] == 'Desempenho') & (teste_df['Pontuacao_Total'] == pontuacao_maxima)]

# Exibindo todos os produtos recomendados com detalhes para leigos
for _, produto in produtos_recomendados.iterrows():
    print(f"Produto recomendado: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']}")
    print("Pontos de vantagem e justificativas:")

    pontuacoes_detalhadas = []
    for col in montante_relacional.index:
        if tipos_dados[col] == 'number':
            pontuacao_atributo = produto[col + '_pontuacao']
            if pontuacao_atributo > 0:
                # Contexto adicional para leigos
                justificativa = (
                    f"O produto teve vantagem no atributo '{col}' com uma pontuação de {pontuacao_atributo:.2f}. "
                    f"Isto significa que o desempenho neste atributo está {pontuacao_atributo * 100:.0f}% mais próximo do ideal ('BOM') "
                    f"em comparação com o mínimo aceitável ('NEUTRO')."
                )
                pontuacoes_detalhadas.append(justificativa)
            elif pontuacao_atributo < 0:
                justificativa = (
                    f"O produto apresentou desvantagem no atributo '{col}' com uma pontuação de {pontuacao_atributo:.2f}. "
                    f"Isto significa que o desempenho neste atributo está {pontuacao_atributo * -100:.0f}% abaixo do mínimo aceitável ('NEUTRO')."
                )
                pontuacoes_detalhadas.append(justificativa)

    # Exibindo as justificativas detalhadas para o produto
    for detalhe in pontuacoes_detalhadas:
        print(detalhe)
    print("\n")

# Gerando gráficos individuais para cada atributo
for col in montante_relacional.index:
    if tipos_dados[col] == 'number':
        fig = go.Figure()

        # Adicionando as barras de pontuação
        for index, row in teste_df.iterrows():
            fig.add_trace(go.Bar(
                y=[row['Modelo']],
                x=[row[col + '_pontuacao']],
                orientation='h',
                name=row['Color'],
                marker_color='red' if row['Color'] == 'BOM' else ('red' if row['Color'] == 'NEUTRO' else 'green'),
                showlegend=False
            ))

        # Adicionando a linha neutra em y=0
        fig.add_hline(y=0, line_color='black', line_dash='dash', annotation_text="Linha Neutra", annotation_position="bottom right")

        # Atualizando layout do gráfico
        fig.update_layout(
            title=f'Pontuação do Atributo: {col}',
            xaxis_title='Pontuação',
            yaxis_title='Modelos',
            showlegend=True
        )

        fig.show()


Produto recomendado: Clevo com pontuação total de 5.446666666666667
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' com uma pontuação de 4.50. Isto significa que o desempenho neste atributo está 450% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' com uma pontuação de 0.95. Isto significa que o desempenho neste atributo está 95% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').


Produto recomendado: Eurocom com pontuação total de 5.446666666666667
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' com uma pontuação de 4.50. Isto significa que o desempenho neste atributo está 450% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' com uma pontuação de 0.95. Isto significa que o desempenho neste atributo está 95% mais próxim

In [ ]:
teste_df.head()

In [ ]:
teste_df = pd.read_csv('base.csv')

# Main df table creation
df_main = pd.DataFrame()
df_main = teste_df

# Drop columns that are not numeric
df_main = df_main.drop(list(df_main.filter(regex='OBJECT')), axis=1)

# Classifiers table
df_neutralGood = pd.DataFrame()
df_neutralGood = teste_df[2:4]

# Calc table
df_calc = pd.DataFrame()
df_calc = teste_df[1:2]

In [ ]:
df_main

## **CÉLULA 2 - DADOS NÃO NUMÉRICOS**

In [ ]:
df = pd.DataFrame()

# Situação/objeto exemplo: compra de um celular
# Começar considerando cores como categoria a ser analisada
# Considerar Vermelho como Muito desejável, Preto como Neutro e Verde como Nada desejável

# A ideia é a criação de diversos métodos de análise e com gráficos a serem analisados
# No caso, cada cor receberia um slider para que o usuário selecione a sua importância
# Considerando valores entre os intervalos dados (trabalhando na base de 0 até 100, logo, números decimais)
# Onde 0 é o mínimo desejável e 1 é o máximo desejável

'''
  Instruções de uso:
    - Com o dataframe já carregado da CÉLULA 1, basta alterar os dados de teste
    (placeholder) aqui presentes e executar o código.
    DICA: Crie cópias da célula, assim execute o código uma vez para cada dado
    qualitativo que desejar comparar.
'''

# Altere aqui! - Início
df['Cores'] = ['Vermelho', 'Verde', 'Branco', 'Preto', 'BOM', 'NEUTRO']
df['Valor Cores'] = [90, 10, 40, 50, 80, 50]
# Altere aqui! 0 Final

df['Tam_scatter'] = [1]*len(df)
symbols = ['diamond-wide']*len(df)

df['Valor X'] = [0]*len(df)

val_neutro = df['Valor Cores'][df.index[df['Cores'] == 'NEUTRO'][0]]
val_bom = df['Valor Cores'][df.index[df['Cores'] == 'BOM'][0]]

df['Definicao'] = pd.Series(dtype='str')
for i in range(len(df)):
    if df['Cores'][i] == 'NEUTRO' or df['Cores'][i] == 'BOM':
        df.loc[i, 'Definicao'] = 'Classificador'
    else:
        df['Definicao'] = 'Desempenho'
# Diversas cores para daltônicos

cores1 = ['red', 'blue']
cores2 = ['pink', 'green']
cores3 = ['orange', 'purple']
cores4 = ['orange', 'gray']

graph = px.scatter(df, x='Valor X', y='Valor Cores', title='Comparação com Indicadores de Desempenho e Classificadores',
                   color='Cores', symbol_sequence=symbols,
                   hover_data={'Cores':False, 'Valor Cores':False, 'Valor X':False,
                               'Valor Referencial': (df['Cores'])})

graph.update_traces(marker={'size': 20})
graph.update_xaxes(fixedrange=True, title=None)
graph.update_yaxes(fixedrange=True, title=None)

for i in range(len(df)):
    if df['Definicao'][i] == 'Classificador' and df['Cores'][i] == 'NEUTRO':
        graph.add_hline(y=df['Valor Cores'][i], line_color=cores3[1], line_width=3, x0=0.3, x1=0.7)
    elif df['Definicao'][i] == 'Classificador' and df['Cores'][i] == 'BOM':
        graph.add_hline(y=df['Valor Cores'][i], line_color=cores3[0], line_width=3, x0=0.3, x1=0.7)


graph.update_xaxes(showticklabels=False, showgrid=False)
graph.update_yaxes(showticklabels=False, showgrid=False)

graph.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=1,
    title_text='Variáveis e Cores'
))

graph.add_vline(x=0, y0=0, y1=1)
graph.add_annotation(
    x=-0.2,
    y=110,
    xref='x',
    yref='y',
    text=f'Linha Neutra<br>(Mínimo Aceitável)',
    showarrow=False,
    font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor=cores3[1],
        opacity=1
)
graph.add_annotation(
    x=0.2,
    y=110,
    xref='x',
    yref='y',
    text=f'Linha Bom<br>(Muito Desejável)',
    showarrow=False,
    font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor=cores3[0],
        opacity=1
)
graph.show()